<a href="https://colab.research.google.com/github/kayleefoor/Foor_DSPN_S24/blob/main/Exercise_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [1]:
library(tidyverse)
install.packages("boot")
library(boot)
install.packages("mlbench")
library(mlbench)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
data(PimaIndiansDiabetes2)
head(PimaIndiansDiabetes2)
df <- PimaIndiansDiabetes2[, !(names(PimaIndiansDiabetes2) %in% "insulin")]
df <- na.omit(df)

,pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,NA,33.6,0.627,50,pos
2,1,85,66,29,NA,26.6,0.351,31,neg
3,8,183,64,NA,NA,23.3,0.672,32,pos
4,1,89,66,23,94,28.1,0.167,21,neg
5,0,137,40,35,168,43.1,2.288,33,pos
6,5,116,74,NA,NA,25.6,0.201,30,neg


In [6]:
dim(df)
head(df)

[1] 532   8

,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [12]:
df_minus_one <- df[-1,]

log_fit <- glm(diabetes~pregnant+glucose+pressure+triceps+mass+pedigree+age,
                data=df_minus_one, family=binomial)
summary(log_fit)

predict(log_fit, newdata = df[1, ], type="response") #prediction is positive because > 0.5

print(df$diabetes[1])

1 
0.729487

[1] pos
Levels: neg pos


So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [15]:
# Initialize `results` data frame
results <- data.frame(
                      actual = df$diabetes,
                      predicted = NA
)
head(results)

#for loop
for (i in 1:nrow(df)){
    # separate individual observation `i` from the rest of your data
    df_minus_i <- df[-i,]

    # train your model
    log_fit <- glm(diabetes~pregnant+glucose+pressure+triceps+mass+pedigree+age,
                data=df_minus_i, family=binomial)

    # test model on hold out observation
    pred <- predict(log_fit, newdata = df[i, ], type="response")

    # classify model prediction as "pos" or "neg" and add to `results`
    if (pred > 0.5){
      results$predicted[i] <- "pos"
    } else {
      results$predicted[i] <- "neg"
    }
}
head(results)


,actual,predicted
,<fct>,<lgl>
1,pos,NA
2,neg,NA
3,neg,NA
4,pos,NA
5,pos,NA
6,pos,NA


,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [17]:
error_rate <- sum(results$actual != results$predicted) / nrow(results)
print(paste("Overall error rate:", error_rate))

[1] "Overall error rate: 0.221804511278195"


# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [19]:
?cv.glm

Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [20]:

cost <- function(r, pi = 0) mean(abs(r - pi) > 0.5)

cv_glm_result <- cv.glm(df,
                        glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
                            data = df, family = binomial),
                        cost = cost, K = nrow(df))

cv_glm_result$delta

$call
cv.glm(data = df, glmfit = glm(diabetes ~ pregnant + glucose + 
    pressure + triceps + mass + pedigree + age, data = df, family = binomial), 
    cost = cost, K = nrow(df))

$K
[1] 532

$delta
[1] 0.2218045 0.2221154

$seed
  [1]       10403          78   203619623  -810653208  1710500911 -1761155592
  [7]  1568123348 -1291009649   855667284   520628531  2058999921   826750906
 [13]  -699026479 -1193019906   373590722 -2079050979 -1476480518  1407193501
 [19]  -221950109  -757701084  -138431237  -278536508 -2067867416  1475822355
 [25] -1979103272   850747119   -17858043  -793313266 -2054972571  1978895378
 [31]  -904321122 -1618746439   358339078 -1898048031  1313665391  2003268208
 [37]   687476103   340540592  1954680684 -2007193449 -1431528228  -457355349
 [43] -1696833143 -1746195550 -1152385143  1923769430  1831420010  -648661659
 [49]  -981756350 -2002046619   899373691  1479999196  -118288797  1791336332
 [55] -1998435152 -1654461109   175530544 -1266194953  -768196339 

How do your results compare to your manual LOOCV above?

> * The error for this model is slightly higher than the manual LOOCV


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [22]:
set.seed(1)

# K = 3
k_3 <- cv.glm(df, glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
           data = df, family = binomial), cost = cost, K = 3)
k_3$delta
# K = 5
k_5 <- cv.glm(df, glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
           data = df, family = binomial), cost = cost, K = 5)
k_5$delta
# K = 10

k_10 <- cv.glm(df, glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
           data = df, family = binomial), cost = cost, K = 10)
k_10$delta
# K = 15
k_15 <- cv.glm(df, glm(diabetes ~ pregnant + glucose + pressure + triceps + mass + pedigree + age,
           data = df, family = binomial), cost = cost, K = 15)
k_15$delta

[1] 0.2105263 0.2168085

[1] 0.2161654 0.2184444

[1] 0.2236842 0.2230942

[1] 0.2274436 0.2281856

#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * As k increases, so too does the error rate

If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> * We would not expect this to occur with the manual LOOCV because it is chronologically iterative, meaning there is no randomization involved in the systemic testing of each observation.


**DUE:** 5pm March 25, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
